# Objective

Discover the data about truck vehicles with following fields:
* Placa
* Marca
* Modelo
* Ano Fab/Ano Modelo
* Chassi
* Números de eixos
* Tração

In [17]:
import pandas as pd
import numpy as np

from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

In [237]:
def crawling_data(plate):
    url = 'http://infocarrosp.com.br/basic-car-info/1387508/'
    return url

In [238]:
def scraping_data(url):
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
    reg_url = url
    req = Request(url=reg_url, headers=headers) 
    html = urlopen(req).read()
    soup = BeautifulSoup(html, 'lxml')
    
    return soup

In [265]:
def transform_data(soup):
    
    #----slicing and split data inputing in variables
    get_city = soup.find_all('td')[1].get_text().split()[0]
    get_estate = soup.find_all('td')[1].get_text().split()[2]
    for row in rows:
        row_td = row.find_all('td')
    get_brand = row_td[0].get_text().split('/')[0].strip()
    get_model = row_td[0].get_text().split('/')[1].strip() + row_td[0].get_text().split('/')[2]
    get_fuel, get_fipe = row_td[1].get_text(), row_td[-1].get_text()
    
    return get_brand, get_model, get_fuel, get_city, get_estate, get_fipe

In [266]:
def export_data(df_list):
    
    from datetime import datetime
    
    time_stampe = (str(datetime.now()).split('.')[0].replace('-',
                '_').replace(' ', '_').replace(':', '')[:-2])
    
    #----create dictionary to output return
    dict_df = {
    'brand':df_list[0], 'model':df_list[1], 'fuel':df_list[2],
    'city':df_list[3], 'estate':df_list[4], 'fipe':df_list[5]
    }
    
    df = pd.DataFrame(dict_df)
    df.to_csv(f'data_col_{time_stampe}', index=False)
    
    return print(f'Sucess Exported_{time_stampe}')

In [282]:
def main():
    list_plates = ['IMU7532']
    brand, model, fuel, city, estate, fipe = [], [], [], [], [], []
    for plate in list_plates:
        url = crawling_data(plate)
        soup = scraping_data(url)
        data = transform_data(soup)

        #----append list
        brand.append(data[0]), model.append(data[1]), fuel.append(data[2])
        city.append(data[3]), estate.append(data[4]), fipe.append(data[5])
        
    return brand, model, fuel, city, estate, fipe

In [284]:
export_data(main())

Sucess Exported_2020_04_26_2217
